In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
c2df = pd.read_csv('C2.txt', sep='\t', header=None, index_col=0)
c2df.head()

In [ ]:
def dist(x, y):
    return np.linalg.norm(x - y)

In [ ]:
def gonzalez(x, k):
    n = x.shape[0]
    c = []
    c.append(x[0])
    fi = [0] * n
    
    for i in range(1, k):
        maxDist = 0
        c.append(x[0])
        for j in range(n):
            currDist = dist(x[j], c[fi[j]])
            if (currDist > maxDist):
                maxDist = currDist
                c[i] = x[j]
        for j in range(n):
            if (dist(x[j], c[fi[j]]) > dist(x[j], c[i])):
                fi[j] = i
    return c, fi

In [ ]:
def center_cost(x, c, fi):
    n = x.shape[0]
    
    maxDist = 0
    maxI = -1
    maxJ = -1
    for i in set(fi):
        center = c[i]
        for j in range(n):
            if (fi[j] != i):
                break
            currDist = dist(center, x[j])
            if (currDist > maxDist):
                maxDist = currDist
                maxI = i
                maxJ = j
    return maxDist

In [ ]:
def mean_cost(x, c, fi):
    n = x.shape[0]
    lst = []
    for i in range(n):
        lst.append(x[i] - c[fi[i]])
    mat = np.array(lst)
    return np.linalg.norm(mat) / n ** (1/2)

In [ ]:
def mean_cost2(x, c, fi):
    n = x.shape[0]
    ssd = 0
    for i in range(n):
        ssd += dist(x[i], c[fi[i]]) ** 2
    return (ssd / n)**(1/2)

In [ ]:
c2 = c2df.as_matrix()
c_gonzalez, fi_gonzalez = gonzalez(c2, 3)

In [ ]:
result_gonzalez = c2df.copy()
result_gonzalez['cluster'] = np.array(fi_gonzalez)

In [ ]:
for i in set(fi_gonzalez):
    cluster = result_gonzalez[result_gonzalez['cluster'] == i]
    plt.scatter(cluster.iloc[:,0], cluster.iloc[:,1])
plt.scatter(x=np.array(c_gonzalez)[:,0], y=np.array(c_gonzalez)[:,1], marker='x', c='black')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
center_cost(c2, c_gonzalez, fi_gonzalez)

In [ ]:
mean_cost(c2, c_gonzalez, fi_gonzalez)

In [ ]:
mean_cost2(c2, c_gonzalez, fi_gonzalez)

In [ ]:
def kmeanspp(x, k):
    n = x.shape[0]
    c = []
    c.append(x[np.random.randint(0,n)])
    fi = [0] * n
    for i in range(1, k):
        lst = []
        for j in range(n):
            lst.append(dist(x[j], c[fi[j]]) ** 2)
        arr = np.array(lst)
        prob = arr / arr.sum()
        idx = np.random.choice(np.arange(n), p=prob)
        c.append(x[idx])
        for j in range(n):
            if (dist(x[j], c[fi[j]]) > dist(x[j], c[i])):
                fi[j] = i
    return c, fi

In [ ]:
c_kmeanspp, fi_kmeanspp = kmeanspp(c2, 3)

In [ ]:
while True:
    c_kmeanspp, fi_kmeanspp = kmeanspp(c2, 3)
    cost = mean_cost(c2, c_kmeanspp, fi_kmeanspp)
    if (cost < 3):
        break

In [ ]:
result_kmeanspp = c2df.copy()
result_kmeanspp['cluster'] = np.array(fi_kmeanspp)

In [ ]:
for i in set(fi_kmeanspp):
    cluster = result_kmeanspp[result_kmeanspp['cluster'] == i]
    plt.scatter(cluster.iloc[:,0], cluster.iloc[:,1])
plt.scatter(x=np.array(c_kmeanspp)[:,0], y=np.array(c_kmeanspp)[:,1], marker='x', c='black')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
kmpp_c = []
kmpp_fi = []
kmpp_cost = []
for i in range(500):
    c_kmeanspp, fi_kmeanspp = kmeanspp(c2, 3)
    kmpp_c.append(c_kmeanspp)
    kmpp_fi.append(fi_kmeanspp)
    kmpp_cost.append(mean_cost(c2, c_kmeanspp, fi_kmeanspp))

In [ ]:
a = plt.hist(kmpp_cost, cumulative=True, bins=100, normed=1)

In [ ]:
x = a[1]
y = np.concatenate((np.array([0]), a[0]))
plt.plot(x, y)
plt.xlabel('3-means cost')
plt.ylabel('CDF')
plt.show()

In [ ]:
def find_center(c, p):
    minDist = np.inf
    minIdx = -1
    for i in range(len(c)):
        currDist = dist(c[i], p)
        if (currDist < minDist):
            minDist = currDist
            minIdx = i
    return minIdx

In [ ]:
def kmeans(x, k, c=[]):
    n = x.shape[0]
    
    if (len(c) == 0):
        for i in range(k):
            c.append(x[i])
    fi = [-1] * n
    
    while True:
        for i in range(n):
            fi[i] = find_center(c, x[i])
        newc = []
        for j in range(k):
            idxs = [idx for idx in range(n) if fi[idx] == j]
            newc.append(x[idxs].mean(axis=0))
        if (np.array_equal(np.array(newc), np.array(c))):
            break
        else:
            c = newc
    
    return c, fi

In [ ]:
c_kmeans1, fi_kmeans1 = kmeans(c2, 3)

In [ ]:
result_kmeans1 = c2df.copy()
result_kmeans1['cluster'] = np.array(fi_kmeans1)

In [ ]:
for i in set(fi_kmeans1):
    cluster = result_kmeans1[result_kmeans1['cluster'] == i]
    plt.scatter(cluster.iloc[:,0], cluster.iloc[:,1])
plt.scatter(x=np.array(c_kmeans1)[:,0], y=np.array(c_kmeans1)[:,1], marker='x', c='black')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
mean_cost(c2, c_kmeans1, fi_kmeans1)

In [ ]:
c_kmeans2, fi_kmeans2 = kmeans(c2, 3, c_gonzalez)

In [ ]:
result_kmeans2 = c2df.copy()
result_kmeans2['cluster'] = np.array(fi_kmeans2)

In [ ]:
for i in set(fi_kmeans2):
    cluster = result_kmeans2[result_kmeans2['cluster'] == i]
    plt.scatter(cluster.iloc[:,0], cluster.iloc[:,1])
plt.scatter(x=np.array(c_kmeans2)[:,0], y=np.array(c_kmeans2)[:,1], marker='x', c='black')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
mean_cost(c2, c_kmeans2, fi_kmeans2)

In [ ]:
c_kmeans3, fi_kmeans3 = kmeans(c2, 3, c_kmeanspp)

In [ ]:
result_kmeans3 = c2df.copy()
result_kmeans3['cluster'] = np.array(fi_kmeans3)

In [ ]:
for i in set(fi_kmeans3):
    cluster = result_kmeans3[result_kmeans3['cluster'] == i]
    plt.scatter(cluster.iloc[:,0], cluster.iloc[:,1])
plt.scatter(x=np.array(c_kmeans3)[:,0], y=np.array(c_kmeans3)[:,1], marker='x', c='black')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
mean_cost(c2, c_kmeans3, fi_kmeans3)

In [ ]:
total = len(kmpp_c)
same = 0

km_cost = []
for i in range(total):
    c_km, fi_km = kmeans(c2, 3, kmpp_c[i])
    km_cost.append(mean_cost(c2, c_km, fi_km))
    if (sum(kmpp_fi[i]) == sum(fi_km)):
        same += 1

same/total

In [ ]:
b = plt.hist(km_cost, cumulative=True, bins=100, normed=1)

In [ ]:
x = b[1]
y = np.concatenate((np.array([0]), b[0]))
plt.plot(x, y)
plt.xlabel('3-means cost')
plt.ylabel('CDF')
plt.show()